# Modelo Logit para Valoración Contingente

En este notebook, estimaremos el modelo Logit utilizando PandasBiogeme, un software estadistico basado en Python para la estimación de modelos de elección. PandasBiogeme también puede ser usado para estimar el modelo dicotómico simple de Valoración Contingente (Logit). PandasBiogeme será también usado para estimar modelos de elección para experimentos de elección.

## Parte 2: Modelo Logit

### Abrir modulos requeridos

El código detallado abajo permite abrir los modulos base para ejecutar el modelo Logit:

- `Pandas`: Permite abrir y realizar operaciones con bases de datos.
- `Biogeme`: Permite estimar modelos de elección

In [ ]:
# Instalar paquetes recomendados
!pip3 install biogeme

In [24]:
# Abrir modulos
import pandas as pd
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta

### Abrir base de datos

La base de datos “clean_energy.csv” contiene información sobre las respuestas de la encuesta de valoración contingente “Clean Energy and You”, conducida en Estados Unidos para conocer las preferencias de los ciudadanos por un programa llamado “Green Choice”, cuyo objetivo es el reemplazo de fuentes fósiles de energía por fuentes renovables, además de la plantación de árboles para la compensación de emisiones de CO2. La pregunta de valoración fue formulada en base a una respuesta dicotómica simple, por lo que cada individuo fue consultado si deseaba pagar un monto ofrecido por los encuestadores.

El codigo descrito a continuación permite abrir la base de datos. La base de datos es guardada en un objeto llamado `df`

In [25]:
df = pd.read_csv('https://raw.githubusercontent.com/ighdez/taller_UNALM/main/taller_vc/clean_energy.csv',index_col=0)
df

,ID,Bid,Choice
0,1,100,0
1,2,5,0
2,3,150,0
3,4,1,1
4,5,5,1
...,...,...,...
199,200,20,1
200,201,30,1
201,202,1,1
202,203,30,1


### Preparar base de datos

El siguiente código prepara la base de datos para Biogeme:

In [26]:
database = db.Database('clean_energy', df)
globals().update(database.variables)

### Definir parametros a estimar

En el siguiente código, definimos los siguientes parametros:

- `alpha`: Parametro del intercepto de la función de utilidad
- `beta`: Parametro que acompaña a `Bid`

En cada parametro se definen las siguientes opciones:

- Nombre del parametro
- Valor inicial para la estimación. Usualmente definido en 0
- Limite inferior. No usado en este caso
- Limite superior. No usado en este caso
- Indicador que toma valor 1 si la variable es fija (no estimada) o 0 si es libre (se debe estimar). Por defecto se define en 0

In [27]:
alpha = Beta('alpha',0,None,None,0)
beta = Beta('beta',0,None,None,0)

### Definir funciones de utilidad

En esta sección, definimos las funciones de utilidad de aceptar y rechazar el monto ofrecido por el entrevistador. Si la persona elige rechazar el monto ofrecido, la utilidad es igual a $V_0 = 0$, mientras que si acepta, la utilidad es igual a:

$$V_1 = \alpha - \beta * Bid$$

El codigo para definir las funciones de utilidad es:

In [28]:
# Crear funciones de utilidad
V0 = 0
V1 = alpha - beta * Bid

# Asociar cada funcion de utilidad al valor que toma la variable Choice para cada caso
V = {0: V0, 1: V1}

# Paso adicional: Definir condiciones de disponibilidad (No usado en este caso, pero necesario en Biogeme, asi que solo rellenamos con 1)
av = {0: 1, 1: 1}

### Definir el modelo Logit y estimar

En este paso, definimos el modelo Logit en base a la información de las funciones de utilidad, las condiciones de disponibilidad, y la variable de elección (`Choice`):

In [29]:
logprob = models.loglogit(V, av, Choice)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'logit'

results = biogeme.estimate()

### Obtener resultados

El siguiente codigo permite ver los resultados de la estimación:

In [30]:
pandasResults = results.getEstimatedParameters()
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
alpha,1.806446,0.249149,7.250450,4.154455e-13,0.237127,7.618038,2.575717e-14
beta,0.019903,0.002947,6.752777,1.450418e-11,0.003077,6.467384,9.971401e-11


### Obtener disposición a pagar media (alpha/beta)

El siguiente codigo permite obtener los estimadores de alpha, beta y la DAP media:

In [31]:
alpha_hat = pandasResults['Value']['alpha']
beta_hat = pandasResults['Value']['beta']

dap = alpha_hat / beta_hat
dap

90.76367675698057